In [50]:
import argparse
import os
import numpy as np
import math
from torchvision.transforms import ToTensor
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [51]:
class Generator(torch.nn.Module):
    
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        super(Generator, self).__init__()
        self._g = torch.nn.Sequential(
            self._g_block(z_dim, hidden_dim),
            self._g_block(hidden_dim, hidden_dim * 2),
            self._g_block(hidden_dim * 2, hidden_dim * 4),
            self._g_block(hidden_dim * 4, hidden_dim * 8),
            nn.Linear(hidden_dim * 8, im_dim),
            nn.Sigmoid()
        )
        
    def _g_block(self, input_dim, output_dim):
        return nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.BatchNorm1d(output_dim),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, noise):
        return self._g(noise)



In [52]:
class Descriminator(torch.nn.Module):
    
    def __init__(self, z_dim=10, im_dim=784, hidden_dim=128):
        super(Descriminator, self).__init__()
        self._g = nn.Sequentuial(
            self._g_block(im_dim, hidden_dim * 4),
            self._g_block(hidden_dim * 4, hidden_dim * 2),
            self._g_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1),
        )
        
    def _d_block(self, input_dim, output_dim):
        return nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, image):
        return self._g(image)

In [53]:
class GAN:
    
    def __init__(self, z_dim, criterion, lr, device):
        self._g = Generator(z_dim).to(device)
        self._g_opt = torch.optim.Adam(self._g_parameters(), lr=lr)
        
        self._d = Discriminator().to(device)
        self._d_opt = torch.optim.Adam(self._d.parameters(), lr=lr)
        
        self._z_dim = z_dim
        self._criteriom = criterion
        self._device = device
        
    def _calc_d_loss(self, batch_real, num_images):
        z = gen_noise(num_images, self._z_dim, self._device)
        fake_samples = self._g(z)
        
        y_fake_ = self._d(fake_samples.detach())
        fake_loss = self._criterion(y_fake_, t.zeros_like(y_fake_))
        
        y_real_ = self._d(batch_real)
        real_loss = self._criterion(y_real_, t.ones_like(y_real_))
        
        _d_loss = (fake_loss + real_loss) / 2
        
        return _d_loss
    
    def _calc_g_loss(self, num_images):
        z = gen_noise(num_images, self._z_dim, self._device)
        fake_samples = self._g(z)
        
        y_ = self._d(fake_samples)
        _g_loss = self._criterion(y_, t.ones_like(y_))
        
        return _g_loss
    
    def _update_step(self, batch_real, flatten_batch=0):
        num_images = len(batch_real)
        if flatten_batch:
            batch_real = batch_real.view(num_images, -1)
        batch_real = batch_real.to(self._device)
        
        self._d_opt.zero_grad()
        d_loss = self._calc_d_loss(batch_real, num_images)
        d_loss.backward(retain_graph=True)
        self._d_opt.step()
        
        self._g_opt.zerograd()
        g_loss = self._calc_g_loss(num_images)
        g_loss.backward()
        self._g_opt.step()
        
        return d_loss.item(), g_loss.item()

In [54]:
class Trainer:
    
    def __init__(self, data_loader, model, batch_size=128, device='cpu'):
        self.batch_size = batch_size
        self.dataloader = data_loader
        self.model = model
    
    def run(self, n_epochs, display_epoch=10, **kwargs):
        update_step = 9
        epoch_update_step = 0
        mean_g_loss = 0
        mean_d_loss = 0
        display_epoch = 10
        device = config['device']
        
        for epoch in tqdm(range(n_epochs)):
            for batch_real, labels in self.dataloader:
                cur_batch_size = len(batch_real)
                
                d_loss, g_loss = self.model._update_step(batch_real,
                                                        labels=labels, **kwargs)

In [55]:
print(torch.__version__)
my_tensor = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.float32, device="cpu")
print(my_tensor)
torch.cuda.is_available()

1.9.1+cpu
tensor([[1., 2., 3.],
        [4., 5., 6.]])


False

In [56]:
batch_size = 128
dataloader = DataLoader(
    datasets.MNIST('.', download=True, transform=ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

config = {
    'criterion': torch.nn.BCEWithLogitsLoss(),
    'z_dim': 64,
    'lr': 0.00001,
    'device': 'cpu'
}
model = GAN(**config)

trainer = Trainer(dataloader, model)
trainer.run(n_epoch=50)

AttributeError: 'GAN' object has no attribute '_g_parameters'